In [1]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("codesignal/lending-club-loan-accepted")


README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

C:\Users\delli\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\delli\.cache\huggingface\hub\datasets--codesignal--lending-club-loan-accepted. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


accepted_2007_to_2018Q4.csv:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

C:\Users\delli\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\csv\csv.py:190: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for batch_idx, df in enumerate(csv_file_reader):
C:\Users\delli\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\csv\csv.py:190: DtypeWarning: Columns (19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  for batch_idx, df in enumerate(csv_file_reader):
C:\Users\delli\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\csv\csv.py:190: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for batch_idx, df in enumerate(csv_file_reader):
C:\Users\delli\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\packaged_modules\csv\csv.py:190: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or se

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
# Preview the first few rows
dataset["train"].to_pandas().head()


In [ ]:
# 📌 1. Install required packages
!pip install -q datasets lightgbm imbalanced-learn pandas scikit-learn

# 📌 2. Load the dataset
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("codesignal/lending-club-loan-accepted")
df = dataset['train'].to_pandas()

# 📌 3. Preprocess data
df = df.drop(columns=['member_id'], errors='ignore')
df = df.dropna(subset=['loan_amnt', 'int_rate', 'term', 'grade', 'emp_title'])

# Convert term to numeric (e.g., '36 months' -> 36)
df['term'] = df['term'].str.extract('(\d+)').astype(float)

# Add a fake target column if not present (REMOVE this if `default_ind` already exists)
import numpy as np
if 'default_ind' not in df.columns:
    df['default_ind'] = np.random.choice([0, 1], size=len(df), p=[0.85, 0.15])  # Simulated target

# Encode categorical variables simply (can be improved later)
df['grade'] = df['grade'].astype('category').cat.codes

# 📌 4. Prepare features and labels
X = df.select_dtypes(include=['float64', 'int64']).drop(columns=['default_ind'])
y = df['default_ind']

# 📌 5. Train-test split + SMOTE
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

# 📌 6. Train LightGBM Classifier
import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(X_res, y_res)

# 📌 7. Evaluate the model
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print("📊 Classification Report:\n")
print(classification_report(y_test, y_pred))
